<a href="https://colab.research.google.com/github/UbaidullahTanoli/LLM-CNN-MLP/blob/main/LLM%2BCNN%2BMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade torch torchvision torchaudio

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d raddar/chest-xrays-indiana-university
!unzip chest-xrays-indiana-university.zip -d /content/dataset/

**0. Importing the Libraries**

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.cuda.amp import GradScaler, autocast
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
import torchvision.transforms as transforms
from torchvision.models import efficientnet_b1, EfficientNet_B1_Weights

**1. Combining Datasets**

In [ ]:
class CombinedDataset(Dataset):
    def __init__(self, reports_csv, proj_csv, image_folder, max_length=256, tokenizer_name="dmis-lab/biobert-base-cased-v1.1", transform=None):
        """
        Args:
            reports_csv (str): Path to the reports CSV (which includes "MeSH", "findings", "impression", and "uid")
            proj_csv (str): Path to the projections CSV (which maps uid to image filename)
            image_folder (str): Folder containing the image files
            max_length (int): Maximum token length for the report text
            tokenizer_name (str): Name of the pretrained tokenizer (BioBERT)
            transform (callable, optional): Image transformation. If None, a default transform is used.
        """
        # Load CSVs and merge on 'uid'
        self.reports_df = pd.read_csv(reports_csv)
        self.proj_df = pd.read_csv(proj_csv)
        self.data = pd.merge(self.reports_df, self.proj_df, on='uid')
        self.image_folder = image_folder

        # Remove rows where both "findings" and "impression" are empty
        self.data = self.data[~(
            (self.data['findings'].isna() | (self.data['findings'].str.strip() == "")) &
            (self.data['impression'].isna() | (self.data['impression'].str.strip() == ""))
        )]

        # Initialize tokenizer for report text
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.max_length = max_length

        # Default image transform if none provided
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
            ])
        else:
            self.transform = transform

        # Prepare labels from "MeSH"
        self.labels = []
        for idx, row in self.data.iterrows():
            mesh_val = str(row['MeSH']).strip().lower()
            label = 0 if mesh_val == 'normal' else 1
            self.labels.append(label)
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # Get image from image folder using filename (assumed to be in column 'filename')
        img_path = os.path.join(self.image_folder, row['filename'])
        image = Image.open(img_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        image = self.transform(image)

        # Concatenate text from "findings" and "impression"
        report_text = ""
        if 'findings' in row and not pd.isna(row['findings']):
            report_text += "Findings: " + str(row['findings']) + " "
        if 'impression' in row and not pd.isna(row['impression']):
            report_text += "Impression: " + str(row['impression'])
        if not report_text.strip():
            report_text = "No report text available."

        # Tokenize the report text
        tokenized = self.tokenizer(
            report_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        input_ids = tokenized['input_ids'].squeeze(0)
        attention_mask = tokenized['attention_mask'].squeeze(0)

        # Get label
        mesh_val = str(row['MeSH']).strip().lower()
        label = 0 if mesh_val == 'normal' else 1

        return {
            'image': image,
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long),
            'uid': row['uid'] if 'uid' in row else -1
        }

    def get_class_distribution(self):
        return dict(zip(*np.unique(self.labels, return_counts=True)))

**2. Combining Models (LLM + CNN + MLP)**

In [ ]:
class LLMCNNMLPClassifier(nn.Module):
    def __init__(self, num_classes=2, freeze_cnn=True, freeze_llm=True,
                 cnn_model_name="efficientnet_b1", llm_model_name="dmis-lab/biobert-base-cased-v1.1"):
        super(LLMCNNMLPClassifier, self).__init__()

        # Load CNN backbone (EfficientNet-B1)
        # Use pretrained weights from torchvision
        weights = EfficientNet_B1_Weights.IMAGENET1K_V1
        self.cnn = efficientnet_b1(weights=weights)
        if freeze_cnn:
            for param in self.cnn.parameters():
                param.requires_grad = False

        # We will use the feature extractor; EfficientNet has an 'extract_features' method.
        # We'll later apply adaptive average pooling to get a vector of size 1280.
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Load LLM backbone (BioBERT)
        self.llm = AutoModel.from_pretrained(llm_model_name)
        if freeze_llm:
            for param in self.llm.parameters():
                param.requires_grad = False

        # Define MLP head that takes concatenated features (1280 from CNN + 768 from LLM = 2048)
        self.mlp = nn.Sequential(
            nn.Linear(2048, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, input_ids, attention_mask, image):
        # CNN branch: extract features from image
        cnn_features = self.cnn.features(image)  # shape: (B, 1280, H, W)
        pooled_cnn = self.avgpool(cnn_features)            # shape: (B, 1280, 1, 1)
        cnn_vector = pooled_cnn.flatten(1) # shape: (B, 1280)

        # LLM branch: get pooled output from BioBERT
        outputs = self.llm(input_ids=input_ids, attention_mask=attention_mask)

        if hasattr(outputs, 'last_hidden_state'):
            llm_vector = outputs.last_hidden_state.mean(dim=1) # shape: (B, 768)
        else:
            # Fallback to pooler_output if last_hidden_state is not available.
            llm_vector = outputs.pooler_output  # shape: (B, 768)
        #llm_vector = outputs.pooler_output                 # shape: (B, 768)

        # Concatenate both feature vectors
        combined = torch.cat((cnn_vector, llm_vector), dim=1)  # shape: (B, 2048)

        # Pass through MLP head
        logits = self.mlp(combined)
        return logits

**3. Computing the Metrics**

In [ ]:
def compute_metrics(labels, preds, probs):
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, zero_division=0)
    rec = recall_score(labels, preds, zero_division=0)
    f1 = f1_score(labels, preds, zero_division=0)
    cm = confusion_matrix(labels, preds)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    try:
        roc_auc = roc_auc_score(labels, probs)
        pr_auc = average_precision_score(labels, probs)
    except:
        roc_auc = 0.0
        pr_auc = 0.0
    return {
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1,
        'specificity': specificity,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc
    }

**4. Training and Evaluating**

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device, scaler=None):
    model.train()
    running_loss = 0.0
    preds_all, labels_all, probs_all = [], [], []

    if scaler is None:
        scaler = torch.amp.GradScaler()

    progress_bar = tqdm(dataloader, desc="Training", dynamic_ncols=True)
    for batch in progress_bar:
        images = batch['image'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        with torch.amp.autocast(device_type='cuda'):
            outputs = model(input_ids, attention_mask, images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * images.size(0)
        probs = torch.softmax(outputs, dim=1)[:, 1]
        _, preds = torch.max(outputs, dim=1)

        preds_all.extend(preds.cpu().numpy())
        labels_all.extend(labels.cpu().numpy())
        probs_all.extend(probs.detach().cpu().numpy())

        progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

    epoch_loss = running_loss / len(dataloader.dataset)
    metrics = compute_metrics(labels_all, preds_all, probs_all)
    metrics['loss'] = epoch_loss
    return metrics

In [ ]:
def eval_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    preds_all, labels_all, probs_all = [], [], []

    with torch.no_grad():
        progress_bar = tqdm(dataloader, desc="Evaluating", dynamic_ncols=True)
        for batch in progress_bar:
            images = batch['image'].to(device)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            with torch.amp.autocast(device_type='cuda'):
                outputs = model(input_ids, attention_mask, images)
                loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            probs = torch.softmax(outputs, dim=1)[:, 1]
            _, preds = torch.max(outputs, dim=1)

            preds_all.extend(preds.cpu().numpy())
            labels_all.extend(labels.cpu().numpy())
            probs_all.extend(probs.detach().cpu().numpy())

            progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

    epoch_loss = running_loss / len(dataloader.dataset)
    metrics = compute_metrics(labels_all, preds_all, probs_all)
    metrics['loss'] = epoch_loss
    return metrics

**5. Training the Model for each Epoch for both the Training and Evaluation Phases**

In [ ]:
def train_and_evaluate(reports_csv, proj_csv, image_folder, num_epochs=40, batch_size=8, learning_rate=1e-3, test_size=0.2, random_state=42):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Create the combined dataset
    full_dataset = CombinedDataset(reports_csv, proj_csv, image_folder)
    class_distribution = full_dataset.get_class_distribution()
    print(f"Class distribution: {class_distribution}")

    split = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)

    for train_index, test_index in split.split(full_dataset, full_dataset.labels):
        train_dataset = torch.utils.data.Subset(full_dataset, train_index)
        test_dataset = torch.utils.data.Subset(full_dataset, test_index)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    # Calculate class weights
    #num_samples = len(train_dataset)
    # Calculate class weights based on the train dataset's class distribution
    train_labels = [full_dataset.labels[i] for i in train_index]
    class_weights = torch.FloatTensor([len(train_labels) / (len(class_distribution) * train_labels.count(label)) for label in class_distribution]).to(device)

    # Initialize the combined model (both branches frozen)
    model = LLMCNNMLPClassifier(num_classes=2, freeze_cnn=True, freeze_llm=True).to(device)

    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.AdamW(model.mlp.parameters(), lr=learning_rate, weight_decay=0.01)
    # Use a LinearLR scheduler over the full num_epochs
    scheduler = optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=num_epochs)
    scaler = torch.amp.GradScaler()

    best_val_metrics = {'f1': 0}

    for epoch in range(1, num_epochs + 1):
        train_metrics = train_epoch(model, train_loader, criterion, optimizer, device, scaler)
        val_metrics = eval_epoch(model, test_loader, criterion, device)

        # Print metrics horizontally in one line
        print(f"Epoch {epoch}/{num_epochs} - "
              f"Train Loss: {train_metrics['loss']:.4f} | Acc: {train_metrics['accuracy']:.4f} | Prec: {train_metrics['precision']:.4f} | Rec: {train_metrics['recall']:.4f} | F1: {train_metrics['f1']:.4f} | Spec: {train_metrics['specificity']:.4f} | ROC AUC: {train_metrics['roc_auc']:.4f} | PR AUC: {train_metrics['pr_auc']:.4f}")
        print(f"                Val Loss: {val_metrics['loss']:.4f} | Acc: {val_metrics['accuracy']:.4f} | Prec: {val_metrics['precision']:.4f} | Rec: {val_metrics['recall']:.4f} | F1: {val_metrics['f1']:.4f} | Spec: {val_metrics['specificity']:.4f} | ROC AUC: {val_metrics['roc_auc']:.4f} | PR AUC: {val_metrics['pr_auc']:.4f}")

        scheduler.step()  # Update learning rate

        if val_metrics['f1'] > best_val_metrics['f1']:
            best_val_metrics = val_metrics
            print(f"  New best model with Val F1: {val_metrics['f1']:.4f}")

    print("\nFinal Validation Results:")
    for metric, value in best_val_metrics.items():
        print(f"  {metric}: {value:.4f}")

    return model, best_val_metrics

In [ ]:
def main():

    reports_csv = "/content/dataset/indiana_reports.csv"
    proj_csv = "/content/dataset/indiana_projections.csv"
    image_folder = "/content/dataset/images/images_normalized"

    train_and_evaluate(reports_csv, proj_csv, image_folder, num_epochs=40, batch_size=8, learning_rate=1e-3)

**6. Running the Model - The real Stuff happens here**

In [ ]:
if __name__ == "__main__":
    main()

Using device: cuda
Class distribution: {np.int64(0): np.int64(2655), np.int64(1): np.int64(4771)}


Training:   0%|          | 0/743 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/186 [00:00<?, ?it/s]

Epoch 1/40 - Train Loss: 0.4835 | Acc: 0.8003 | Prec: 0.8585 | Rec: 0.8252 | F1: 0.8415 | Spec: 0.7556 | ROC AUC: 0.8701 | PR AUC: 0.9212
                Val Loss: 0.2789 | Acc: 0.8910 | Prec: 0.9232 | Rec: 0.9058 | F1: 0.9144 | Spec: 0.8644 | ROC AUC: 0.9531 | PR AUC: 0.9723
  New best model with Val F1: 0.9144


Training:   0%|          | 0/743 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/186 [00:00<?, ?it/s]

Epoch 39/40 - Train Loss: 0.4165 | Acc: 0.9384 | Prec: 0.9563 | Rec: 0.9473 | F1: 0.9518 | Spec: 0.9223 | ROC AUC: 0.9769 | PR AUC: 0.9822
                Val Loss: 0.4455 | Acc: 0.9455 | Prec: 0.9689 | Rec: 0.9455 | F1: 0.9571 | Spec: 0.9454 | ROC AUC: 0.9774 | PR AUC: 0.9807


Training:   0%|          | 0/743 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/186 [00:00<?, ?it/s]

Epoch 40/40 - Train Loss: 0.5369 | Acc: 0.9242 | Prec: 0.9371 | Rec: 0.9455 | F1: 0.9413 | Spec: 0.8861 | ROC AUC: 0.9664 | PR AUC: 0.9722
                Val Loss: 0.4894 | Acc: 0.9307 | Prec: 0.9723 | Rec: 0.9183 | F1: 0.9445 | Spec: 0.9529 | ROC AUC: 0.9759 | PR AUC: 0.9797

Final Validation Results:
  accuracy: 0.9509
  precision: 0.9623
  recall: 0.9613
  f1: 0.9618
  specificity: 0.9322
  roc_auc: 0.9763
  pr_auc: 0.9793
  loss: 0.3296
